This Python code defines and utilizes the `Data_generator` class to generate synthetic data for a physics experiment. The program then cleans up the data by removing outliers and subsequently stores the clean data in a pickle file. 

## Data Generation

`Data_generator` is the main part of this script and is initialized with two parameters: the number of events (`numevents`) and a boolean indicating whether the generated data should be normalized or not. The synthetic data is a representation of the result of physics events involving multiple particles, where each particle is characterized by different properties such as `eta`, `mass`, `phi`, `pt`, `charge`, and `genPartFlav`.

Upon initialization, `Data_generator` configures a series of functions and input variables, which are then utilized in the `generate_fake_data` method to generate synthetic data. 

The `generate_fake_data` method starts by creating a dictionary, `data`, with keys for each variable and empty lists as values. It then generates random data for these variables using relevant statistical distributions which are based on the physical properties being simulated. 

After the data has been generated, `Data_generator` includes an optional step of renaming and reordering keys in the dictionary in a more standard format, before returning the data.

## Data Cleaning

After generating the data, outliers are removed using the `remove_outliers` function. This function uses the limits defined in a YAML file to identify and remove the outliers in the data. 

## Data Storage

Once the outliers have been removed, the data is then stored as a pickle file at a specified location for later use.



In [11]:
import pickle
import numpy as np
from copy import deepcopy
import sys
sys.path.append('../utils/')
from DD_data_extractor_git import Data_generator
import matplotlib.pyplot as plt
import pandas as pd
import yaml
import os

In [12]:
num_events = 2000000
Data_generator1 = Data_generator(num_events, normalize=True)
data_dict=Data_generator1.getData()

Using 19 workers


Applying functions: 50it [01:49,  2.19s/it]


In [13]:
print(len(data_dict['1_phi']))
print(data_dict['1_phi'][:10])

2000000
[-0.22677017  2.16314675 -2.36792007 -3.05590973 -2.26387487 -0.989072
  1.79703848  3.02768574 -2.17145962 -0.18144251]


In [14]:
reference_length = len(next(iter(data_dict.values())))
for feature_name, values in data_dict.items():
    if len(values) != reference_length:
       print(f"Feature '{feature_name}' has {len(values)} values, expected {reference_length}.")


In [15]:
def compute_percentiles_from_pickle(filename):
    with open(filename, 'rb') as f:
        raw_data_dict = pickle.load(f)
    
    numeric_data_dict = {k: v for k, v in raw_data_dict.items() if (k not in dontremove_outliers) and np.issubdtype(type(v[0]), np.number)}

    # Compute the required percentiles for each numeric feature
    lower_percentiles = {k: np.percentile(v, 0.03) for k, v in numeric_data_dict.items()}
    upper_percentiles = {k: np.percentile(v, 99.7) for k, v in numeric_data_dict.items()}

    return lower_percentiles, upper_percentiles

def remove_outliers(data, lower_percentiles, upper_percentiles):
    outlier_mask = np.zeros(len(next(iter(data.values()))), dtype=bool)
    for feature_name, values in data.items():
        if (feature_name not in dontremove_outliers) and (feature_name in lower_percentiles):
            lower_value = lower_percentiles[feature_name]
            upper_value = upper_percentiles[feature_name]
            feature_outlier_mask = (np.array(values) < lower_value) | (np.array(values) > upper_value)
            outlier_mask |= feature_outlier_mask  # update the outlier mask
    
    # Remove rows with outliers from all features in the data dictionary
    cleaned_data = {k: np.array(v)[~outlier_mask].tolist() for k, v in data.items()}
    return cleaned_data

base_path = os.path.dirname(os.getcwd())
raw_data_pickle_file = os.path.join(base_path, 'saved_files', 'extracted_data', 'TEST10_data_Aug3')
dontremove_outliers=['event', 'genWeight', 'MET_phi', '1_phi', '1_genPartFlav', '2_phi', '2_genPartFlav', '3_phi', '3_genPartFlav', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3']
lower_percentiles, upper_percentiles = compute_percentiles_from_pickle(raw_data_pickle_file)

data_dictcopy = deepcopy(data_dict)
data_dict_removed_outliers2 = remove_outliers(data_dictcopy, lower_percentiles, upper_percentiles)

print(data_dict_removed_outliers2.keys())
print(len(data_dict_removed_outliers2['mass_12']))

dict_keys(['event', 'genWeight', 'MET_phi', '1_phi', '1_genPartFlav', '2_phi', '2_genPartFlav', '3_phi', '3_genPartFlav', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'ma

In [10]:
print(data_dict_removed_outliers2['mass_12'][:30])
print(type(data_dict_removed_outliers2))
df=pd.DataFrame(data_dict_removed_outliers2)
print(df.head())

[222.91184415029005, 94.67291038905961, 332.31630771867475, 142.60057981314216, 66.3575279707401, 196.29616791155817, 282.3419453930044, 284.8550325761306, 195.4833241103937, 162.43686873496284, 85.46821201493785, 146.1409672672796, 175.63286117525166, 100.78825555197156, 151.46961410296717, 98.87408091809667, 148.84611376017648, 81.40417501052714, 40.99751822809804, 74.76296397811251, 129.99882358746353, 137.8105922999336, 159.05824064317542, 230.639832393913, 133.0559658989557, 92.00831021210304, 117.25428916799096, 23.14147945460032, 112.02488074812263, 140.6992989773038]
<class 'dict'>
   event  genWeight   MET_phi     1_phi  1_genPartFlav     2_phi  \
0   8297   0.595798 -2.374731 -0.226770              1  2.792510   
1   3159  -0.903095  1.383645  2.163147              1  1.070950   
2   5570  -0.432716  0.963632 -2.367920              3  0.083417   
3    203  -0.756514 -1.090332 -2.263875              1  2.124214   
4   5513  -0.906506  2.771565  1.797038              3 -0.70878

In [7]:
base_path = os.path.dirname(os.getcwd())
folder = "fake_data"

full_folder_path = os.path.join(base_path,"saved_files", folder)

os.makedirs(full_folder_path, exist_ok=True)

filename = "Aug17_10mil.pkl"

full_file_path = os.path.join(full_folder_path, filename)

with open(full_file_path, 'wb') as f:
    pickle.dump(data_dict_removed_outliers2, f)


print(data_dict_removed_outliers2.keys())
print(len(data_dict_removed_outliers2['mass_12']))

dict_keys(['event', 'genWeight', 'MET_phi', '1_phi', '1_genPartFlav', '2_phi', '2_genPartFlav', '3_phi', '3_genPartFlav', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'ma